# Deep Learning models using Keras

Keras is a user-friendly neural network library written in Python. We will build a regression model to predict an employee's wage per hour.

Note: The datasets we will be using are relatively clean, so we will not perform any data preprocessing in order to get our data ready for modeling. Datasets that you will use in future projects may not be so clean - for example, they may have missing values - so you may need to use data preprocessing techniques to alter your datasets to get more accurate results.

# Regression Model using Keras

For our regression deep learning model, the first step is to read in the data we will use as input. For this example, we are using the 'hourly wages' dataset. To start, we will use Pandas to read in the data. I will not go into detail on Pandas, but it is a library you should become familiar with if you're looking to dive further into data science and machine learning.

'df' stands for dataframe. Pandas read in the CSV file as a dataframe. The 'head()' function will show the first 5 rows of the dataframe so you can check that the data has been read in properly and can take an initial look at how the data is structured.

In [2]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [3]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/IntroToDNNwKeras.git cloned-repo
%cd cloned-repo

Cloning into 'cloned-repo'...
remote: Enumerating objects: 361, done.
remote: Counting objects: 100% (229/229), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 361 (delta 143), reused 109 (delta 66), pack-reused 132
Receiving objects: 100% (361/361), 12.00 MiB | 14.10 MiB/s, done.
Resolving deltas: 100% (193/193), done.
/content/cloned-repo


In [22]:
#read in training data
wage_df = pd.read_csv('/content/cloned-repo/hourly_wage')

#view data structure
wage_df.head()
print(wage_df.shape)


(534, 10)


Next, we need to split up our dataset into inputs (train_X) and our target (train_y). Our input will be every column except 'wage_per_hour' because 'wage_per_hour' is what we will be attempting to predict. Therefore, 'wage_per_hour' will be our target.

We will use the pandas 'drop' function to drop the column 'wage_per_hour' from our dataframe and store it in the variable 'train_X'. This will be our input.

In [23]:
#create a dataframe with all training data except the target column
wage_data = wage_df.drop(columns=['wage_per_hour'])

#check that the target variable has been removed
wage_data.head()

,union,education_yrs,experience_yrs,age,female,marr,south,manufacturing,construction
0,0,8,21,35,1,1,0,1,0
1,0,9,42,57,1,1,0,1,0
2,0,12,1,19,0,0,0,1,0
3,0,12,4,22,0,0,0,0,0
4,0,12,17,35,0,1,0,0,0


In [25]:
#create a dataframe with only the target column
wage_y = wage_df[['wage_per_hour']]

#view dataframe
wage_y.head()

,wage_per_hour
0,5.10
1,4.95
2,6.67
3,4.00
4,7.50


In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    wage_data, wage_y, test_size=0.20)

The model type that we will be using is Sequential. Sequential is the easiest way to build a model in Keras. It allows you to build a model layer by layer. Each layer has weights that correspond to the layer the follows it.

We use the 'add()' function to add layers to our model. We will add two layers and an output layer.

'Dense' is the layer type. Dense is a standard layer type that works for most cases. In a dense layer, all nodes in the previous layer connect to the nodes in the current layer.

We have 10 nodes in each of our input layers. This number can also be in the hundreds or thousands. Increasing the number of nodes in each layer increases model capacity. I will go into further detail about the effects of increasing model capacity shortly.

'Activation' is the activation function for the layer. An activation function allows models to take into account nonlinear relationships. For example, if you are predicting diabetes in patients, going from age 10 to 11 is different than going from age 60â??61.

The activation function we will be using is ReLU or Rectified Linear Activation. Although it is two linear pieces, it has been proven to work well in neural networks.

The first layer needs an input shape. The input shape specifies the number of rows and columns in the input. The number of columns in our input is stored in 'n_cols'. There is nothing after the comma which indicates that there can be any amount of rows.

The last layer is the output layer. It only has one node, which is for our prediction.

Next, we need to compile our model. Compiling the model takes two parameters: optimizer and loss.

The optimizer controls the learning rate. We will be using 'Adam' as our optimizer. Adam is generally a good optimizer to use for many cases. The Adam optimizer adjusts the learning rate throughout training.

The learning rate determines how fast the optimal weights for the model are calculated. A smaller learning rate may lead to more accurate weights (up to a certain point), but the time it takes to compute the weights will be longer.

For our loss function, we will use 'mean_squared_error'. It is calculated by taking the average squared difference between the predicted and actual values. It is a popular loss function for regression problems. The closer to 0 this is, the better the model performed.



In [27]:
#create model
model = Sequential()

#get number of columns in training data
n_cols = X_train.shape[1]

#add model layers
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

#compile model using mse as a measure of model performance
model.compile(optimizer='adam', loss='mean_squared_error')

#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)

Now we will train our model. To train, we will use the 'fit()' function on our model with the following five parameters: training data (train_X), target data (train_y), validation split, the number of epochs and callbacks.

The validation split will randomly split the data into use for training and testing. During training, we will be able to see the validation loss, which gives the mean squared error of our model on the validation set. We will set the validation split at 0.2, which means that 20% of the training data we provide in the model will be set aside for testing model performance.

The number of epochs is the number of times the model will cycle through the data. The more epochs we run, the more the model will improve, up to a certain point. After that point, the model will stop improving during each epoch. In addition, the more epochs, the longer the model will take to run. To monitor this, we will use 'early stopping'.

Early stopping will stop the model from training before the number of epochs is reached if the model stops improving. We will set our early stopping monitor to 3. This means that after 3 epochs in a row in which the model doesn't improve, training will stop. Sometimes, the validation loss can stop improving then improve in the next epoch, but after 3 epochs in which the validation loss doesn't improve, it usually won't improve again.

In [28]:
#train model
model.fit(X_train, y_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])

Epoch 1/30
11/11 [==============================] - 1s 24ms/step - loss: 102.0816 - val_loss: 62.4066
Epoch 2/30
11/11 [==============================] - 0s 6ms/step - loss: 87.2989 - val_loss: 49.5178
Epoch 3/30
11/11 [==============================] - 0s 6ms/step - loss: 71.9368 - val_loss: 38.4583
Epoch 4/30
11/11 [==============================] - 0s 8ms/step - loss: 59.1226 - val_loss: 29.8462
Epoch 5/30
11/11 [==============================] - 0s 6ms/step - loss: 48.3272 - val_loss: 22.6461
Epoch 6/30
11/11 [==============================] - 0s 6ms/step - loss: 38.4081 - val_loss: 17.0830
Epoch 7/30
11/11 [==============================] - 0s 6ms/step - loss: 30.5349 - val_loss: 15.1742
Epoch 8/30
11/11 [==============================] - 0s 6ms/step - loss: 27.0021 - val_loss: 17.2404
Epoch 9/30
11/11 [==============================] - 0s 4ms/step - loss: 26.6690 - val_loss: 17.8238
Epoch 10/30
11/11 [==============================] - 0s 4ms/step - loss: 26.5638 - val_loss: 16.85

Making predictions on new data

If you want to use this model to make predictions on new data, we would use the 'predict()' function, passing in our new data.
The output would be 'wage_per_hour' predictions.

In [64]:
wage_pred = model.predict(X_test)

4/4 [==============================] - 0s 3ms/step


In [71]:
print("prediction", "\tground truth")
for i in range(len(y_test_wages)):
  print(wage_pred[i],"\t", y_test_wages[i])


prediction 	ground truth
[11.363782] 	 [5.5]
[7.3762746] 	 [11.]
[9.510837] 	 [8.56]
[9.035282] 	 [11.11]
[8.246197] 	 [5.77]
[11.0579405] 	 [17.25]
[6.5139503] 	 [4.15]
[7.8230605] 	 [12.5]
[12.811257] 	 [22.83]
[8.20562] 	 [5.]
[9.47067] 	 [8.9]
[10.382978] 	 [14.29]
[8.813756] 	 [13.45]
[8.797692] 	 [16.26]
[9.174382] 	 [5.]
[8.726514] 	 [3.8]
[9.031005] 	 [3.6]
[6.2663517] 	 [2.01]
[9.678316] 	 [8.2]
[9.444367] 	 [10.43]
[8.169385] 	 [6.75]
[8.448624] 	 [5.]
[8.462085] 	 [5.35]
[6.8821335] 	 [3.]
[9.873268] 	 [17.86]
[9.264361] 	 [3.5]
[6.052514] 	 [4.5]
[9.473007] 	 [7.3]
[9.130155] 	 [9.56]
[7.7929635] 	 [4.75]
[9.109382] 	 [4.17]
[9.47717] 	 [11.25]
[7.5711193] 	 [9.15]
[8.945449] 	 [4.25]
[10.405208] 	 [12.]
[8.4914255] 	 [11.32]
[9.393583] 	 [8.4]
[12.108306] 	 [10.2]
[6.226786] 	 [2.85]
[9.110693] 	 [4.]
[9.311604] 	 [11.02]
[10.65536] 	 [15.]
[8.9719305] 	 [13.]
[8.062501] 	 [10.]
[10.317361] 	 [16.]
[9.029799] 	 [7.65]
[6.749594] 	 [5.]
[8.635253] 	 [20.4]
[7.2173085] 	 [6.

As you increase the number of nodes and layers in a model, the model capacity increases. Increasing model capacity can lead to a more accurate model, up to a certain point, at which the model will stop improving. Generally, the more training data you provide, the larger the model should be. We are only using a tiny amount of data, so our model is pretty small. The larger the model, the more computational capacity it requires and it will take longer to train.

Let's create a new model using the same training data as our previous model. This time, we will add a layer and increase the nodes in each layer to 200. We will train the model to see if increasing the model capacity will improve our validation score.

In [68]:
#training a new model on the same data to show the effect of increasing model capacity

#create model
model_mc = Sequential()

#add model layers
model_mc.add(Dense(200, activation='relu', input_shape=(n_cols,)))
model_mc.add(Dense(200, activation='relu'))
model_mc.add(Dense(200, activation='relu'))
model_mc.add(Dense(1))

#compile model using mse as a measure of model performance
model_mc.compile(optimizer='adam', loss='mean_squared_error')

In [69]:
#train model
model_mc.fit(train_X, train_y, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])

Epoch 1/30
14/14 [==============================] - 1s 16ms/step - loss: 34.0046 - val_loss: 30.5651
Epoch 2/30
14/14 [==============================] - 0s 5ms/step - loss: 21.0113 - val_loss: 30.9176
Epoch 3/30
14/14 [==============================] - 0s 6ms/step - loss: 19.8901 - val_loss: 35.9667
Epoch 4/30
14/14 [==============================] - 0s 7ms/step - loss: 21.1847 - val_loss: 30.8216


In [70]:
wage_pred = model_mc.predict(X_test)

4/4 [==============================] - 0s 3ms/step


In [72]:
print("prediction", "\tground truth")
for i in range(len(y_test_wages)):
  print(wage_pred[i],"\t", y_test_wages[i])


prediction 	ground truth
[11.363782] 	 [5.5]
[7.3762746] 	 [11.]
[9.510837] 	 [8.56]
[9.035282] 	 [11.11]
[8.246197] 	 [5.77]
[11.0579405] 	 [17.25]
[6.5139503] 	 [4.15]
[7.8230605] 	 [12.5]
[12.811257] 	 [22.83]
[8.20562] 	 [5.]
[9.47067] 	 [8.9]
[10.382978] 	 [14.29]
[8.813756] 	 [13.45]
[8.797692] 	 [16.26]
[9.174382] 	 [5.]
[8.726514] 	 [3.8]
[9.031005] 	 [3.6]
[6.2663517] 	 [2.01]
[9.678316] 	 [8.2]
[9.444367] 	 [10.43]
[8.169385] 	 [6.75]
[8.448624] 	 [5.]
[8.462085] 	 [5.35]
[6.8821335] 	 [3.]
[9.873268] 	 [17.86]
[9.264361] 	 [3.5]
[6.052514] 	 [4.5]
[9.473007] 	 [7.3]
[9.130155] 	 [9.56]
[7.7929635] 	 [4.75]
[9.109382] 	 [4.17]
[9.47717] 	 [11.25]
[7.5711193] 	 [9.15]
[8.945449] 	 [4.25]
[10.405208] 	 [12.]
[8.4914255] 	 [11.32]
[9.393583] 	 [8.4]
[12.108306] 	 [10.2]
[6.226786] 	 [2.85]
[9.110693] 	 [4.]
[9.311604] 	 [11.02]
[10.65536] 	 [15.]
[8.9719305] 	 [13.]
[8.062501] 	 [10.]
[10.317361] 	 [16.]
[9.029799] 	 [7.65]
[6.749594] 	 [5.]
[8.635253] 	 [20.4]
[7.2173085] 	 [6.



---



---



# Classification Model using Keras

Now let's move on to building our model for classification. Since many steps will be a repeat from the previous model, I will only go over new concepts.

For this next model, we are going to predict if patients have diabetes or not.

In [12]:
#read in training data
train_df_2 = pd.read_csv('diabetes.csv')

#view data structure
train_df_2.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [13]:
#create a dataframe with all training data except the target column
train_X_2 = train_df_2.drop(columns=['Outcome'])

#check that the target variable has been removed
train_X_2.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


When separating the target column, we need to call the 'to_categorical()' function so that column will be 'one-hot encoded'. Currently, a patient with no diabetes is represented with a 0 in the diabetes column and a patient with diabetes is represented with a 1. 

With one-hot encoding, the integer will be removed and a binary variable is inputted for each category. In our case, we have two categories: no diabetes and diabetes.

A patient with no diabetes will be represented by [1 0] and a patient with diabetes will be represented by [0 1].

In [14]:
#one-hot encode target column
train_y_2 = to_categorical(train_df_2.Outcome)

#vcheck that target column has been converted
train_y_2[0:5]

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

The last layer of our model has 2 nodes - one for each option: the patient has diabetes or they don't.

The activation is 'softmax'. Softmax makes the output sum up to 1 so the output can be interpreted as probabilities. The model will then make its prediction based on which option has a higher probability.

We will use 'categorical_crossentropy' for our loss function. This is the most common choice for classification. A lower score indicates that the model is performing better.

To make things even easier to interpret, we will use the 'accuracy' metric to see the accuracy score on the validation set at the end of each epoch.

In [15]:
#create model
model_2 = Sequential()

#get number of columns in training data
n_cols_2 = train_X_2.shape[1]

#add layers to model
model_2.add(Dense(250, activation='relu', input_shape=(n_cols_2,)))
model_2.add(Dense(250, activation='relu'))
model_2.add(Dense(250, activation='relu'))
model_2.add(Dense(2, activation='softmax'))

#compile model using accuracy to measure model performance
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
#train model
model_2.fit(train_X_2, train_y_2, epochs=30, validation_split=0.2, callbacks=[early_stopping_monitor])

Epoch 1/30
20/20 [==============================] - 1s 16ms/step - loss: 4.3017 - accuracy: 0.5961 - val_loss: 1.2936 - val_accuracy: 0.6364
Epoch 2/30
20/20 [==============================] - 0s 6ms/step - loss: 0.7351 - accuracy: 0.6564 - val_loss: 0.7386 - val_accuracy: 0.5584
Epoch 3/30
20/20 [==============================] - 0s 7ms/step - loss: 0.8268 - accuracy: 0.6254 - val_loss: 1.1644 - val_accuracy: 0.6494
Epoch 4/30
20/20 [==============================] - 0s 6ms/step - loss: 0.9612 - accuracy: 0.6140 - val_loss: 1.2030 - val_accuracy: 0.5000
Epoch 5/30
20/20 [==============================] - 0s 7ms/step - loss: 0.9688 - accuracy: 0.6221 - val_loss: 0.6427 - val_accuracy: 0.6234
Epoch 6/30
20/20 [==============================] - 0s 6ms/step - loss: 0.5846 - accuracy: 0.6971 - val_loss: 0.7009 - val_accuracy: 0.6558
Epoch 7/30
20/20 [==============================] - 0s 6ms/step - loss: 0.5740 - accuracy: 0.7036 - val_loss: 0.6365 - val_accuracy: 0.6494
Epoch 8/30
20/20 [=